# Getting started

In this notebook, we'll walk through how to use the PolicyEngine.py package to run simulations and produce analyses. We'll start with a basic analysis in the UK that doesn't use any databases, and then start saving and loading things into a database.

## Basic analysis

To start, let's run through a simulation of the UK, and create a chart of the distribution of household income.

In [15]:
from policyengine.models import Simulation, Aggregate, policyengine_uk_model, policyengine_uk_latest_version
from policyengine.utils.datasets import create_uk_dataset

uk_dataset = create_uk_dataset()


sim = Simulation(
    dataset=uk_dataset,
    model=policyengine_uk_model,
    model_version=policyengine_uk_latest_version,
)

sim.run()

income_ranges = [0, 20000, 40000, 60000, 80000, 100000, 150000, 200000, 300000, 500000, 1_000_000]
aggregates = []
for i in range(len(income_ranges) - 1):
    aggregates.append(
        Aggregate(
            entity="household",
            variable_name="hbai_household_net_income",
            aggregate_function="count",
            filter_variable_name="hbai_household_net_income",
            filter_variable_geq=income_ranges[i],
            filter_variable_leq=income_ranges[i + 1],
            simulation=sim,
        )
    )

aggregates = Aggregate.run(aggregates)

In [18]:
import plotly.express as px

px.bar(
    x=list(map(str, income_ranges[:-1])),
    y=[agg.value for agg in aggregates],
)